# 内存和 IO 接口

本节将向您介绍 PyArrow 内存管理和 IO 系统中的主要概念：

- 缓冲区
- 内存池
- 类文件和流式对象

## 引用和分配内存

### {class}`pyarrow.Buffer`

{class}`~pyarrow.Buffer` 对象封装了 C++ 的 {cpp:class}`arrow::Buffer` 类型，这是 Apache Arrow 在 C++ 中进行内存管理的主要工具。它允许更高级别的数组类安全地与它们可能拥有或可能不拥有的内存进行交互。{cpp:class}`arrow::Buffer` 可以进行零拷贝切片，以允许 Buffers 廉价地引用其他 Buffers，同时保持内存生命周期和清晰的父子关系。

{cpp:class}`arrow::Buffer` 有许多实现，但它们都提供了标准接口：数据指针和长度。这与 Python 的内置缓冲区协议和 {class}`memoryview` 对象类似。

可以通过调用 {func}`~pyarrow.py_buffer` 函数从任何实现了缓冲区协议的 Python 对象创建 {class}`~pyarrow.Buffer`。

考虑字节对象：

In [1]:
import pyarrow as pa

data = b'abcdefghijklmnopqrstuvwxyz'

buf = pa.py_buffer(data)

buf.size, buf

(26,
 <pyarrow.Buffer address=0x7fc2242ade50 size=26 is_cpu=True is_mutable=False>)

以这种方式创建 {class}`~pyarrow.Buffer` 不会分配任何内存；它是对从数据字节对象导出的内存的零拷贝视图。

外部内存，以原始指针和大小的形式，也可以使用 {class}`~pyarrow.foreign_buffer` 函数进行引用。

在需要 Python 缓冲区或 {class}`memoryview` 的情况下，可以使用 Buffers，并且这种转换是零拷贝的：

In [2]:
memoryview(buf)

{class}`~pyarrow.Buffer` 的 {meth}`~pyarrow.Buffer.to_pybytes` 方法将 {class}`~pyarrow.Buffer` 的数据转换为 Python 字节字符串（因此会复制数据）：

In [3]:
buf.to_pybytes()

b'abcdefghijklmnopqrstuvwxyz'

### 内存池
所有内存分配和释放（如 C 中的 `malloc` 和 `free`）都在 {class}`pyarrow.MemoryPool` 的一个实例中进行跟踪。这意味着我们可以精确地跟踪已分配的内存量：

In [4]:
pa.total_allocated_bytes()

0

从默认池分配一个可调整大小的 {class}`pyarrow.Buffer`：

In [5]:
buf = pa.allocate_buffer(1024, resizable=True)
pa.total_allocated_bytes()

1024

In [6]:
buf.resize(2048)

pa.total_allocated_bytes()

2048

默认分配器以最小 64 字节的增量请求内存。如果缓冲区被垃圾回收，所有内存都会被释放：

In [7]:
buf = None

pa.total_allocated_bytes()

0

除了默认的内置内存池外，根据 Arrow 是如何构建的，还可以选择其他内存池（如 [mimalloc](https://github.com/microsoft/mimalloc)）。可以获取内存池的后端名称：

In [8]:
pa.default_memory_pool().backend_name

'jemalloc'

````{seealso}
- [内存池文档](https://arrow.apache.org/docs/python/api/memory.html#api-memory-pool)
````

## 输入和输出

Arrow C++ 库为不同种类的 IO 对象提供了几个抽象接口：

- 只读流
- 支持随机访问的只读文件
- 只写流
- 支持随机访问的只写文件
- 支持读写和随机访问的文件